In [9]:
import json
import urllib.request

import requests
import os
import env

def getMetaCards(card_ids):
    responses = []
    for card_id in card_ids:
        url = "http://metabase.jabama.com/api/card/" + card_id + "/query"

        headers = {
            'X-Metabase-Session': "daabf2cf-c6f5-413f-9cb3-e1bffad2d699",
            'Cache-Control': "no-cache",
            'Postman-Token': "4914830f-7ad3-4213-8738-1133e80d1c9b"
            }

        responses.append(requests.request("POST", url, headers=headers))    
    return responses

def getDataFromResponses(responses):
    responses_data = []
    for response in responses:
        cols = json.loads(response.text)['data']['cols']
        rows = json.loads(response.text)['data']['rows']

        col_titles = []
        for col in cols:
            col_titles.append(col['name'])

        responses_data.append({
            "columns": col_titles,
            "rows": rows
        })
    return responses_data

def formatMessagesText(response_tables):
    messages_text = []
    for response_table in response_tables:
        plain_text = ""
        response_table.get("columns").pop(0)
        for row in response_table.get("rows"):
            indx = 1
            for col in response_table.get("columns"):
                plain_text = plain_text + str(row[0]) + ", " + str(col) + ": " + str(row[indx]) + os.linesep
                indx = indx + 1
        messages_text.append(plain_text)
    return messages_text

def getUserIDByEmail(emails):
    responses = []
    for email in emails:  
        querystring = {
            "email" : email
        }
        headers = {
            'Content-Type': "application/json",
            'Authorization': "Bearer " + env.METABOT_TOKEN,
            'User-Agent': "PostmanRuntime/7.20.1",
            'Accept': "*/*",
            'Cache-Control': "no-cache",
            'Postman-Token': "1eed9ef6-4ab9-44d9-bf2a-650df7574c84,7e6fc185-e1fd-4ca8-9806-782d0de54a84",
            'Host': "slack.com",
            'Accept-Encoding': "gzip, deflate",
            'Connection': "keep-alive",
            'cache-control': "no-cache"
            }
        response = requests.request("GET", env.SLACK_API_LOOK_UP_BY_EMAIL, headers=headers, params=querystring)
        responses.append(json.loads(response.text)['user']['id'])
    return responses
        
def sendMessagesToSlackUsers(user_ids, messages_text):
    responses = []
    for user_id in user_ids:
        msg_indx = 0
        for message_text in messages_text:  
            payload = {
            "channel" : user_id,
            "text" : message_text,
            "as_user": True
            }
            headers = {
                'Content-Type': "application/json",
                'Authorization': "Bearer " + env.METABOT_TOKEN,
                'Cache-Control': "no-cache",
                'Postman-Token': "3dea07a8-cf24-47cb-b8cd-2a141a41a0b3"
                }
            response = requests.request("POST", env.SLACK_API_POST_MESSAGE, data=str(payload), headers=headers)
            responses.append({
                'user_id': user_id,
                'message_index': msg_indx,
                'status': json.loads(response.text)['ok']
            })
            msg_indx = msg_indx + 1
    return responses

metacard_responses = getMetaCards([env.CARD_ID_ACCOMMODATION, env.CARD_ID_TOTAL])
responses_data = getDataFromResponses(metacard_responses)
messages_text = formatMessagesText(responses_data)

user_ids = getUserIDByEmail([env.EMAIL_NEGAR])
send_messages_status = sendMessagesToSlackUsers(user_ids, messages_text)

print(send_messages_status)

non-Promoters, OrderCount: 13
non-Promoters, Voucher: 4
non-Promoters, TFO/TO: 31 %
Total, OrderCount: 13
Total, Voucher: 4
Total, TFO/TO: 31 %

Accomodation, OrderCount: 13
Accomodation, Voucher: 4
Accomodation, TFO/TO: 31 %
Hotel (Jabama Only), OrderCount: 20
Hotel (Jabama Only), Voucher: 10
Hotel (Jabama Only), TFO/TO: 50 %
Total, OrderCount: 33
Total, Voucher: 14
Total, TFO/TO: 42 %

None
